In [1]:
! pip install -U FlagEmbedding[finetune]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 3.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 99.9 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00:00:0100:

## Fine-tune

Below are the arguments for fine-tuning:

The following arguments are for model:
- `model_name_or_path`: The model checkpoint for initialization.
- `config_name`: Pretrained config name or path if not the same as model_name.
- `tokenizer_name`: Pretrained tokenizer name or path if not the same as model_name.
- `cache_dir`: Where do you want to store the pre-trained models downloaded from s3.
- `trust_remote_code`: Trust remote code
- `token`: The token to use when accessing the model.

The following arguments are for data:
- `train_data`: One or more paths to training data. `query: str`, `pos: List[str]`, `neg: List[str]` are required in the training data. Argument type: multiple.
- `cache_path`: Where do you want to store the cached data.
- `train_group_size`: (No metadata provided)
- `query_max_len`: The maximum total input sequence length after tokenization for passage. Sequences longer than this will be truncated.
- `passage_max_len`: The maximum total input sequence length after tokenization for passage. Sequences longer than this will be truncated.
- `pad_to_multiple_of`: If set will pad the sequence to be a multiple of the provided value.
- `max_example_num_per_dataset`: The max number of examples for each dataset.
- `query_instruction_for_retrieval`: Instruction for query.
- `query_instruction_format`: Format for query instruction.
- `knowledge_distillation`: Use knowledge distillation when `pos_scores: List[float]` and `neg_scores: List[float]` are in features of training data.
- `passage_instruction_for_retrieval`: Instruction for passage.
- `passage_instruction_format`: Format for passage instruction.
- `shuffle_ratio`: The ratio of shuffling the text.
- `same_dataset_within_batch`: All samples in the same batch comes from the same dataset.
- `small_threshold`: The threshold of small dataset. All small dataset in the same directory will be merged into one dataset.
- `drop_threshold`: The threshold for dropping merged small dataset. If the number of examples in the merged small dataset is less than this threshold, it will be dropped.

And the following extra arguments:
- `negatives_cross_device`: Share negatives across devices.
- `temperature`: Temperature used for similarity score.
- `fix_position_embedding`: Freeze the parameters of position embeddings.
- `sentence_pooling_method`: The pooling method. Available options: cls, mean, last_token. Default: cls.
- `normalize_embeddings`: Whether to normalize the embeddings.
- `sub_batch_size`: Sub batch size for training.
- `kd_loss_type`: The loss type for knowledge distillation. Available options: kl_div, m3_kd_loss. Default: kl_div.

In [2]:
import wandb

wandb.login(key="5075b85e4e708e828d33dba7fcc413dcdecbe4c8")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tringo1507 (tringooo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# 1) Gỡ sạch flash-attn
!pip uninstall -y flash-attn

# 2) Thiết biến môi trường để tắt import Flash-Attention
%env HF_NO_FLASH_ATTN=1


Found existing installation: flash_attn 2.8.1
Uninstalling flash_attn-2.8.1:
  Successfully uninstalled flash_attn-2.8.1
env: HF_NO_FLASH_ATTN=1


In [9]:
%%bash
torchrun --nproc_per_node 2 \
  -m FlagEmbedding.finetune.reranker.encoder_only.base \
  --model_name_or_path BAAI/bge-reranker-v2-m3 \
  --cache_dir /kaggle/working/cache/model \
  --train_data /kaggle/input/ft-data/training.json \
  --cache_path /kaggle/working/cache/data \
  --train_group_size 12 \
  --query_max_len 512 \
  --passage_max_len 1024 \
  --pad_to_multiple_of 8 \
  --query_instruction_for_rerank 'Với vai trò là một chuyên gia pháp luật, hãy tìm kiếm các điều khoản, quy định pháp luật có liên quan trực tiếp đến vấn đề: ' \
  --query_instruction_format '{}{}' \
  --knowledge_distillation False \
  --output_dir test_encoder_only_base_bge-reranker-v2-m3 \
  --overwrite_output_dir \
  --learning_rate 1e-5 \
  --bf16 \
  --num_train_epochs 20 \
  --per_device_train_batch_size 4 \
  --gradient_accumulation_steps 16 \
  --dataloader_drop_last True \
  --warmup_ratio 0.1 \
  --deepspeed /kaggle/input/ft-data/ds_stage0.json \
  --logging_steps 1 \
  --save_steps 500 \
  --gradient_checkpointing \
  --save_total_limit 3 \
  --report_to wandb \
  --run_name "finetune-bge-$(date +%Y%m%d_%H%M)"


Process is terminated.


In [ ]:
# ! pip install -U datasets pytrec_eval FlagEmbedding


In [ ]:
# from datasets import load_dataset

# queries = load_dataset("json", data_files="/kaggle/input/ft-data/test_queries.jsonl")["train"]
# corpus = load_dataset("json", data_files="/kaggle/input/ft-data/corpus.jsonl")["train"]
# qrels = load_dataset("json", data_files="/kaggle/input/ft-data/test_qrels.jsonl")["train"]

# queries_text = queries["text"]
# corpus_text = [text for sub in corpus["text"] for text in sub]

In [ ]:
# qrels_dict = {}
# for line in qrels:
#     if line['qid'] not in qrels_dict:
#         qrels_dict[line['qid']] = {}
#     qrels_dict[line['qid']][line['docid']] = line['relevance']

In [ ]:
# !pip install faiss-cpu

In [ ]:
# import faiss
# import numpy as np
# from tqdm import tqdm


# def search(model, queries_text, corpus_text):
    
#     queries_embeddings = model.encode_queries(queries_text)
#     corpus_embeddings = model.encode_corpus(corpus_text)
    
#     # create and store the embeddings in a Faiss index
#     dim = corpus_embeddings.shape[-1]
#     index = faiss.index_factory(dim, 'Flat', faiss.METRIC_INNER_PRODUCT)
#     corpus_embeddings = corpus_embeddings.astype(np.float32)
#     index.train(corpus_embeddings)
#     index.add(corpus_embeddings)
    
#     query_size = len(queries_embeddings)

#     all_scores = []
#     all_indices = []

#     # search top 100 answers for all the queries
#     for i in tqdm(range(0, query_size, 32), desc="Searching"):
#         j = min(i + 32, query_size)
#         query_embedding = queries_embeddings[i: j]
#         score, indice = index.search(query_embedding.astype(np.float32), k=100)
#         all_scores.append(score)
#         all_indices.append(indice)

#     all_scores = np.concatenate(all_scores, axis=0)
#     all_indices = np.concatenate(all_indices, axis=0)
    
#     # store the results into the format for evaluation
#     results = {}
#     for idx, (scores, indices) in enumerate(zip(all_scores, all_indices)):
#         results[queries["id"][idx]] = {}
#         for score, index in zip(scores, indices):
#             if index != -1:
#                 results[queries["id"][idx]][corpus["id"][index]] = float(score)
                
#     return results

In [ ]:
# from FlagEmbedding.abc.evaluation.utils import evaluate_metrics, evaluate_mrr
# from FlagEmbedding import FlagModel

# k_values = [10,100]

# raw_name = "BAAI/bge-reranker-v2-m3"
# finetuned_path = "/kaggle/working/test_encoder_only_base_bge-reranker-v2-m3"

In [ ]:
# raw_model = FlagModel(
#     raw_name, 
#     query_instruction_for_retrieval="Với vai trò là một chuyên gia pháp luật, hãy tìm kiếm các điều khoản, quy định pháp luật có liên quan trực tiếp đến vấn đề: ",
#     devices=[0],
#     use_fp16=False
# )

# results = search(raw_model, queries_text, corpus_text)

# eval_res = evaluate_metrics(qrels_dict, results, k_values)
# mrr = evaluate_mrr(qrels_dict, results, k_values)

# for res in eval_res:
#     print(res)
# print(mrr)

In [ ]:
# ft_model = FlagModel(
#     finetuned_path, 
#     query_instruction_for_retrieval="Với vai trò là một chuyên gia pháp luật, hãy tìm kiếm các điều khoản, quy định pháp luật có liên quan trực tiếp đến vấn đề: ",
#     devices=[0],
#     use_fp16=False
# )

# results = search(ft_model, queries_text, corpus_text)

# eval_res = evaluate_metrics(qrels_dict, results, k_values)
# mrr = evaluate_mrr(qrels_dict, results, k_values)

# for res in eval_res:
#     print(res)
# print(mrr)